# LLM for query optimization

The problem:
- Singularity 6 / Palia: We should search Reddit for Palia
- Akili Interactive / EndeavorRx: We should search Reddit for EndeavorRx
- Rad.ai Omni: We should search Reddit for Rad.ai Omni



In [1]:
from core import Seed, init

init()

In [2]:
seed = Seed.init("Singularity 6", "Palia", "singularity6.com")

queries = [
    f'site:reddit.com "{seed.company}" "{seed.product}"',
    f'site:reddit.com "{seed.product}"',
]

queries

['site:reddit.com "Singularity 6" "Palia"', 'site:reddit.com "Palia"']

In [3]:
from google_search import search

query_results = [list(search(query, num=10)) for query in queries]
query_results

2024-09-17 10:04:34.385 | DEBUG    | google_search:search:58 - Google search results: {'kind': 'customsearch#search', 'url': {'type': 'application/json', 'template': 'https://www.googleapis.com/customsearch/v1?q={searchTerms}&num={count?}&start={startIndex?}&lr={language?}&safe={safe?}&cx={cx?}&sort={sort?}&filter={filter?}&gl={gl?}&cr={cr?}&googlehost={googleHost?}&c2coff={disableCnTwTranslation?}&hq={hq?}&hl={hl?}&siteSearch={siteSearch?}&siteSearchFilter={siteSearchFilter?}&exactTerms={exactTerms?}&excludeTerms={excludeTerms?}&linkSite={linkSite?}&orTerms={orTerms?}&dateRestrict={dateRestrict?}&lowRange={lowRange?}&highRange={highRange?}&searchType={searchType}&fileType={fileType?}&rights={rights?}&imgSize={imgSize?}&imgType={imgType?}&imgColorType={imgColorType?}&imgDominantColor={imgDominantColor?}&alt=json'}, 'queries': {'request': [{'title': 'Google Custom Search - site:reddit.com "Singularity 6" "Palia"', 'totalResults': '580', 'searchTerms': 'site:reddit.com "Singularity 6" "P

[[SearchResult(title='Palia developers, Singularity 6, axes 35% of staff just after Steam ...', link='https://www.reddit.com/r/MMORPG/comments/1bz2e0z/palia_developers_singularity_6_axes_35_of_staff/', snippet='Apr 9, 2024 ... Palia developers, Singularity 6, axes 35% of staff just after Steam launch ... Palia isn\'t quite an "MMO" but it\'s still news. The developers have\xa0...', formattedUrl='https://www.reddit.com/.../palia_developers_singularity_6_axes_35_of_staf...'),
  SearchResult(title='Cozy MMO Palia Developer Singularity 6 Has Suffered Layoffs : r ...', link='https://www.reddit.com/r/pcgaming/comments/1bwiuin/cozy_mmo_palia_developer_singularity_6_has/', snippet='Apr 5, 2024 ... Cozy MMO Palia Developer Singularity 6 Has Suffered Layoffs ... The game is just not very good. For a life Sim it lacks basic features, the\xa0...', formattedUrl='https://www.reddit.com/r/.../cozy_mmo_palia_developer_singularity_6_has...'),
  SearchResult(title='More layoffs at S6, teams down to a fe

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

from pydantic import BaseModel, Field
from typing import List

class RatedResult(BaseModel):
    title: str = Field(description="Title of the search result")
    link: str = Field(description="Link to the search result")
    snippet: str = Field(description="Snippet of the search result")

    product_relevance: int = Field(description="Relevance of the search result for product feedback and critique (1 is least relevant, 5 is most relevant)", ge=1, le=5)
    company_relevance: int = Field(description="Relevance of the search result to the company critique (1 is least relevant, 5 is most relevant)", ge=1, le=5)

class RatedResults(BaseModel):
    rated_queries: List[RatedResult] = Field(description="List of relevance-rated search results")

_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
Review a list of search results and rate their relevance to A) the company {company} and B) the product {product} made by {company} on a 1-5 scale.
            """,
        ),
        (
            "human",
            """
Search results:
{search_results}
            """,
        ),
    ]
)



llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
runnable = _prompt | llm.with_structured_output(RatedResults)
llm_result = runnable.invoke(
    {
        "company": seed.company,
        "product": seed.product,
        "search_results": query_results[0] + query_results[1]
    }
)

llm_result

RatedResults(rated_queries=[RatedResult(title='Palia developers, Singularity 6, axes 35% of staff just after Steam ...', link='https://www.reddit.com/r/MMORPG/comments/1bz2e0z/palia_developers_singularity_6_axes_35_of_staff/', snippet='Apr 9, 2024 ... Palia developers, Singularity 6, axes 35% of staff just after Steam launch ... Palia isn\'t quite an "MMO" but it\'s still news. The developers have ...', product_relevance=4, company_relevance=5), RatedResult(title='Cozy MMO Palia Developer Singularity 6 Has Suffered Layoffs : r ...', link='https://www.reddit.com/r/pcgaming/comments/1bwiuin/cozy_mmo_palia_developer_singularity_6_has/', snippet='Apr 5, 2024 ... Cozy MMO Palia Developer Singularity 6 Has Suffered Layoffs ... The game is just not very good. For a life Sim it lacks basic features, the ...', product_relevance=4, company_relevance=5), RatedResult(title='More layoffs at S6, teams down to a few dozen people now. : r/Palia', link='https://www.reddit.com/r/Palia/comments/1ctgcw1/m

In [7]:
for rated_result in llm_result.rated_queries:
    print(f"""
# [{rated_result.title}]({rated_result.link})
{rated_result.snippet}

Product relevance: {rated_result.product_relevance}
Company relevance: {rated_result.company_relevance}

""")



# [Palia developers, Singularity 6, axes 35% of staff just after Steam ...](https://www.reddit.com/r/MMORPG/comments/1bz2e0z/palia_developers_singularity_6_axes_35_of_staff/)
Apr 9, 2024 ... Palia developers, Singularity 6, axes 35% of staff just after Steam launch ... Palia isn't quite an "MMO" but it's still news. The developers have ...

Product relevance: 4
Company relevance: 5



# [Cozy MMO Palia Developer Singularity 6 Has Suffered Layoffs : r ...](https://www.reddit.com/r/pcgaming/comments/1bwiuin/cozy_mmo_palia_developer_singularity_6_has/)
Apr 5, 2024 ... Cozy MMO Palia Developer Singularity 6 Has Suffered Layoffs ... The game is just not very good. For a life Sim it lacks basic features, the ...

Product relevance: 4
Company relevance: 5



# [More layoffs at S6, teams down to a few dozen people now. : r/Palia](https://www.reddit.com/r/Palia/comments/1ctgcw1/more_layoffs_at_s6_teams_down_to_a_few_dozen/)
May 16, 2024 ... A small company like Singularity 6 that offers a FREE

In [10]:
import numpy as np

print(f"""
Number of search results: {len(llm_result.rated_queries)}
Average product relevance: {np.mean([rated_result.product_relevance for rated_result in llm_result.rated_queries]):.1f}
Average company relevance: {np.mean([rated_result.company_relevance for rated_result in llm_result.rated_queries]):.1f}
""")


Number of search results: 19
Average product relevance: 3.2
Average company relevance: 3.1



In [12]:
print("""# Product-relevant search results\n\n""")

for rated_result in sorted(llm_result.rated_queries, key=lambda x: x.product_relevance, reverse=True)[:10]:
    print(f"""
## [{rated_result.title}]({rated_result.link})
{rated_result.snippet}

Product relevance: {rated_result.product_relevance}
Company relevance: {rated_result.company_relevance}

""")

print("""# Company-relevant search results\n\n""")
for rated_result in sorted(llm_result.rated_queries, key=lambda x: x.company_relevance, reverse=True)[:10]:
    print(f"""
## [{rated_result.title}]({rated_result.link})
{rated_result.snippet}

Product relevance: {rated_result.product_relevance}
Company relevance: {rated_result.company_relevance}

""")


# Product-relevant search results



## [Palia](https://www.reddit.com/r/Palia/)
... Palia: Welcome to the Palia subreddit! Palia is a cozy community ... Palia content creators and artists, moderators, Shepps, and Singularity 6 employees.

Product relevance: 5
Company relevance: 5



## [We are Singularity 6, developers and publishers of Palia. Ask Us ...](https://www.reddit.com/r/Palia/comments/1650bkg/we_are_singularity_6_developers_and_publishers_of/)
Aug 30, 2023 ... We are Singularity 6, developers and publishers of Palia. Ask Us Anything! Official Post.

Product relevance: 5
Company relevance: 5



## [GENUINE PALIA FEEDBACK (PROS AND CONS/) : r/Palia](https://www.reddit.com/r/Palia/comments/18nadl4/genuine_palia_feedback_pros_and_cons/)
Dec 21, 2023 ... Cons- ((Meaning: Please work on these palia devs!)) The temple of the winds was not fun and a nightmare. I spent 2 days hours looking in the ...

Product relevance: 5
Company relevance: 3



## [Palia developers, Singularity 6, a